# NYC Parking Tickets: Violation Rate Prediction

In [104]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

from datetime import datetime, timedelta

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# importing libraries

import csv
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [105]:
selected_columns = [
    'Street Code1',
    'Street Code2',
    'Street Code3',
    'Street Name',
    # 'Violation Location',
    # 'Violation Post Code',
    # 'Intersecting Street',
    'Violation Precinct',
    'Violation County',
    # 'Vehicle Body Type',
    'Issue Date',
    # 'Date First Observed',
    'Violation Time',
    #'Days Parking In Effect',
    #'From Hours In Effect',
    #'To Hours In Effect',
    # 'Violation Code',
]

'''
Genel olarak, "county" (ilçe) daha büyük bir alanı temsil eder. Bir "county", genellikle belirli bir eyaletin sınırları içinde bulunan büyük bir coğrafi bölgeyi kapsar. Bir eyalette birden çok "county" bulunabilir ve her biri kendi yerel yönetimine sahiptir.

Öte yandan, "precinct" (seçim bölgesi) daha küçük bir coğrafi bölgeyi ifade eder. Bu, genellikle seçmenlerin oy kullandığı belirli bir mahalle veya köyü temsil eder. Seçim bölgesi, seçimlerde kullanılan ve oy verme işlemini düzenleyen bir kavramdır.
'''

file_path = 'all_data_for_one_street.csv'
df = pd.read_csv(file_path, usecols=selected_columns)
df

,Issue Date,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name
0,06/20/2017,6430,5880,72630,78,1204P,K,13TH STREET
1,03/09/2017,6430,0,0,78,0941A,K,13TH STREET
2,04/08/2017,6430,5430,5580,78,1109A,K,13TH STREET
3,09/23/2016,6430,5880,72630,78,1151A,K,13TH STREET
4,10/18/2016,6430,40404,40404,78,0227P,K,13TH STREET
...,...,...,...,...,...,...,...,...
14806,09/11/2023,6430,5580,5680,78,0911A,K,13TH STREET
14807,09/11/2023,6430,5430,5580,78,0918A,K,13TH STREET
14808,09/11/2023,6430,5430,5580,78,0918A,K,13TH STREET
14809,09/14/2023,6430,5280,5430,78,0945A,K,13TH STREET


In [106]:
df['Violation Precinct'].isna().sum()

0

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14811 entries, 0 to 14810
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Issue Date          14811 non-null  object
 1   Street Code1        14811 non-null  int64 
 2   Street Code2        14811 non-null  int64 
 3   Street Code3        14811 non-null  int64 
 4   Violation Precinct  14811 non-null  int64 
 5   Violation Time      14811 non-null  object
 6   Violation County    14766 non-null  object
 7   Street Name         14811 non-null  object
dtypes: int64(4), object(4)
memory usage: 925.8+ KB


In [107]:
df['Violation Time']

0        1204P
1        0941A
2        1109A
3        1151A
4        0227P
         ...  
14806    0911A
14807    0918A
14808    0918A
14809    0945A
14810    0950A
Name: Violation Time, Length: 14811, dtype: object

In [108]:
df_1 = df.copy()

In [109]:
df_1[df_1['Issue Date'].str.contains('AM')]

,Issue Date,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name
6085,06/13/2019 12:00:00 AM,6430,5130,5280,78,0308A,K,13TH STREET
6086,06/17/2019 12:00:00 AM,6430,5130,5280,78,0313A,K,13TH STREET
6087,06/20/2019 12:00:00 AM,6430,5680,5780,78,1245A,K,13TH STREET
6088,06/20/2019 12:00:00 AM,6430,5580,5680,78,0119P,K,13TH STREET
6089,06/21/2019 12:00:00 AM,6430,5130,5280,78,0315A,K,13TH STREET
...,...,...,...,...,...,...,...,...
7751,06/22/2020 12:00:00 AM,6430,5680,5780,78,1254P,K,13TH STREET
7752,06/23/2020 12:00:00 AM,6430,5430,5580,78,0420P,K,13TH STREET
7753,06/24/2020 12:00:00 AM,6430,0,0,78,0308P,K,13TH STREET
7754,06/24/2020 12:00:00 AM,6430,5580,5680,78,0126P,K,13TH STREET


In [110]:
df_1['Issue Date'] = df_1['Issue Date'].str.split(' ').str[0]

In [7]:
# import sweetviz as sv
# analyze_report = sv.analyze(df_1)
# analyze_report.show_html('analyze.html', open_browser=False)

In [111]:
df_1

,Issue Date,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation Time,Violation County,Street Name
0,06/20/2017,6430,5880,72630,78,1204P,K,13TH STREET
1,03/09/2017,6430,0,0,78,0941A,K,13TH STREET
2,04/08/2017,6430,5430,5580,78,1109A,K,13TH STREET
3,09/23/2016,6430,5880,72630,78,1151A,K,13TH STREET
4,10/18/2016,6430,40404,40404,78,0227P,K,13TH STREET
...,...,...,...,...,...,...,...,...
14806,09/11/2023,6430,5580,5680,78,0911A,K,13TH STREET
14807,09/11/2023,6430,5430,5580,78,0918A,K,13TH STREET
14808,09/11/2023,6430,5430,5580,78,0918A,K,13TH STREET
14809,09/14/2023,6430,5280,5430,78,0945A,K,13TH STREET


In [112]:
def convert_to_24hr_format(time_str):
    try:
        if '.' in time_str:
            return None
        if time_str[-1] == 'P':
            return '{:02d}:{:02d}'.format((int(time_str[:2]) + 12) % 24, int(time_str[2:4]))
        elif time_str[-1] == 'A':
            return '{:02d}:{:02d}'.format(int(time_str[:2]) % 12, int(time_str[2:4]))
        else:
            return None
    except (ValueError, TypeError):
        return None

df_1['Violation Time'] = df_1['Violation Time'].astype(str)
df_1.loc[:, 'Violation Time'] = df_1['Violation Time'].apply(convert_to_24hr_format)
df_1 = df_1.dropna(subset=['Violation Time'])

In [113]:
df_1['Violation Time']

0        00:04
1        09:41
2        11:09
3        11:51
4        14:27
         ...  
14806    09:11
14807    09:18
14808    09:18
14809    09:45
14810    09:50
Name: Violation Time, Length: 14811, dtype: object

In [114]:
df_1['Violation Datetime'] = pd.to_datetime(df_1['Issue Date'] + ' ' + df_1['Violation Time'], format='%m/%d/%Y %H:%M')

In [115]:
df_1['Violation Datetime']

0       2017-06-20 00:04:00
1       2017-03-09 09:41:00
2       2017-04-08 11:09:00
3       2016-09-23 11:51:00
4       2016-10-18 14:27:00
                ...        
14806   2023-09-11 09:11:00
14807   2023-09-11 09:18:00
14808   2023-09-11 09:18:00
14809   2023-09-14 09:45:00
14810   2023-09-08 09:50:00
Name: Violation Datetime, Length: 14811, dtype: datetime64[ns]

In [116]:
df_1['Violation Datetime (Month)'] = df_1['Violation Datetime'].dt.month
df_1['Violation Datetime (Day)'] = df_1['Violation Datetime'].dt.day
df_1['Violation Datetime (Year)'] = df_1['Violation Datetime'].dt.year
df_1['Violation Datetime (Hour)'] = df_1['Violation Datetime'].dt.hour
df_1 = df_1.drop(columns=['Issue Date', 'Violation Time'])

In [117]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14811 entries, 0 to 14810
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Street Code1                14811 non-null  int64         
 1   Street Code2                14811 non-null  int64         
 2   Street Code3                14811 non-null  int64         
 3   Violation Precinct          14811 non-null  int64         
 4   Violation County            14766 non-null  object        
 5   Street Name                 14811 non-null  object        
 6   Violation Datetime          14811 non-null  datetime64[ns]
 7   Violation Datetime (Month)  14811 non-null  int32         
 8   Violation Datetime (Day)    14811 non-null  int32         
 9   Violation Datetime (Year)   14811 non-null  int32         
 10  Violation Datetime (Hour)   14811 non-null  int32         
dtypes: datetime64[ns](1), int32(4), int64(4), object(2)
me

In [118]:
df_1

,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation County,Street Name,Violation Datetime,Violation Datetime (Month),Violation Datetime (Day),Violation Datetime (Year),Violation Datetime (Hour)
0,6430,5880,72630,78,K,13TH STREET,2017-06-20 00:04:00,6,20,2017,0
1,6430,0,0,78,K,13TH STREET,2017-03-09 09:41:00,3,9,2017,9
2,6430,5430,5580,78,K,13TH STREET,2017-04-08 11:09:00,4,8,2017,11
3,6430,5880,72630,78,K,13TH STREET,2016-09-23 11:51:00,9,23,2016,11
4,6430,40404,40404,78,K,13TH STREET,2016-10-18 14:27:00,10,18,2016,14
...,...,...,...,...,...,...,...,...,...,...,...
14806,6430,5580,5680,78,K,13TH STREET,2023-09-11 09:11:00,9,11,2023,9
14807,6430,5430,5580,78,K,13TH STREET,2023-09-11 09:18:00,9,11,2023,9
14808,6430,5430,5580,78,K,13TH STREET,2023-09-11 09:18:00,9,11,2023,9
14809,6430,5280,5430,78,K,13TH STREET,2023-09-14 09:45:00,9,14,2023,9


In [119]:
df_1.sort_values(by=['Violation Datetime'], inplace=True)
df_1

,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation County,Street Name,Violation Datetime,Violation Datetime (Month),Violation Datetime (Day),Violation Datetime (Year),Violation Datetime (Hour)
1836,0,0,0,78,K,13TH STREET,2016-06-22 09:34:00,6,22,2016,9
1844,6430,5280,5430,78,K,13TH STREET,2016-06-22 09:34:00,6,22,2016,9
1840,0,0,0,78,K,13TH STREET,2016-06-22 09:37:00,6,22,2016,9
1852,6430,5280,5430,78,K,13TH STREET,2016-06-22 13:03:00,6,22,2016,13
1849,6430,5130,5280,78,K,13TH STREET,2016-06-28 03:31:00,6,28,2016,3
...,...,...,...,...,...,...,...,...,...,...,...
14646,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:09:00,9,28,2023,3
14647,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:11:00,9,28,2023,3
14648,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:13:00,9,28,2023,3
14649,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:15:00,9,28,2023,3


In [162]:
import pandas as pd

def calculate_location_violation_rate(df, location, precinct, start_time, end_time):

    one_week_before = start_time - pd.DateOffset(weeks=1)

    filtered_df = len(df[(df['Street Name'] == location) & (df['Violation Precinct'] == precinct) & (df['Violation Datetime'] >= start_time) & (df['Violation Datetime'] <= end_time)])
    print("filtered_df:", filtered_df)

    total_records = len(df[(df['Violation Datetime'] >= one_week_before) & (df['Violation Datetime'] <= end_time)])
    print("total_records:", total_records)

    if total_records > 0:
        violation_rate = (filtered_df / total_records) * 100
    else:
        violation_rate = 0.0
    return violation_rate

location = '13TH STREET'
location_precinct = 78
violation_datetime = pd.to_datetime('2016-07-11 08:49:00')
estimating_time = pd.to_datetime('2016-07-11 09:59:00')

rate = calculate_location_violation_rate(df_1, location, location_precinct, violation_datetime, estimating_time)
print(f'Location Violation Rate for {location} between {violation_datetime} and {estimating_time}: {rate:.2f}%')

filtered_df: 7
total_records: 36
Location Violation Rate for 13TH STREET between 2016-07-11 08:49:00 and 2016-07-11 09:59:00: 19.44%


In [163]:
def calculate_location_violation_rate(row, df):
    location = row['Street Name']
    precinct = row['Violation Precinct']
    start_time = row['Violation Datetime']
    end_time = row['Violation Datetime'] + pd.DateOffset(hours=1)  # Assuming 1 hour interval

    one_week_before = start_time - pd.DateOffset(weeks=1)

    filtered_df = len(df[(df['Street Name'] == location) & (df['Violation Precinct'] == precinct) & (df['Violation Datetime'] >= start_time) & (df['Violation Datetime'] <= end_time)])

    total_records = len(df[(df['Violation Datetime'] >= one_week_before) & (df['Violation Datetime'] <= end_time)])

    if total_records == 0:
        return 0.0

    if total_records > 0:
        violation_rate = (filtered_df / total_records) * 100
    else:
        violation_rate = 0.0

    return violation_rate

df_1['Violation Rate'] = df_1.apply(calculate_location_violation_rate, df=df_1, axis=1)

In [164]:
df_1['Violation Rate']

1836     100.000000
1844     100.000000
1840      33.333333
1852      25.000000
1849      20.000000
            ...    
14646      9.803922
14647      7.843137
14648      5.882353
14649      3.921569
14650      1.960784
Name: Violation Rate, Length: 14811, dtype: float64

In [165]:
df_1

,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation County,Street Name,Violation Datetime,Violation Datetime (Month),Violation Datetime (Day),Violation Datetime (Year),Violation Datetime (Hour),Violation Rate
1836,0,0,0,78,K,13TH STREET,2016-06-22 09:34:00,6,22,2016,9,100.000000
1844,6430,5280,5430,78,K,13TH STREET,2016-06-22 09:34:00,6,22,2016,9,100.000000
1840,0,0,0,78,K,13TH STREET,2016-06-22 09:37:00,6,22,2016,9,33.333333
1852,6430,5280,5430,78,K,13TH STREET,2016-06-22 13:03:00,6,22,2016,13,25.000000
1849,6430,5130,5280,78,K,13TH STREET,2016-06-28 03:31:00,6,28,2016,3,20.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
14646,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:09:00,9,28,2023,3,9.803922
14647,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:11:00,9,28,2023,3,7.843137
14648,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:13:00,9,28,2023,3,5.882353
14649,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:15:00,9,28,2023,3,3.921569


In [169]:
df_1['Violation Rate'].max()

100.0

In [166]:
def scale_between_0_and_1(value, min_value, max_value):
    """
    Scale a value to be between 0 and 1.

    :param value: The value to be scaled.
    :param min_value: The minimum value in the range of your data.
    :param max_value: The maximum value in the range of your data.
    :return: Scaled value between 0 and 1.
    """
    if max_value == min_value:
        return 0  # Avoid division by zero if all values are the same

    return (value - min_value) / (max_value - min_value)

In [167]:
df_1['Violation Rate (Scaled)'] = df_1['Violation Rate'].apply(lambda x: scale_between_0_and_1(x, df_1['Violation Rate'].min(), df_1['Violation Rate'].max()))
df_1['Violation Rate (Scaled)']

1836     1.000000
1844     1.000000
1840     0.327869
1852     0.243852
1849     0.193443
           ...   
14646    0.090646
14647    0.070878
14648    0.051109
14649    0.031340
14650    0.011572
Name: Violation Rate (Scaled), Length: 14811, dtype: float64

In [170]:
df_1

,Street Code1,Street Code2,Street Code3,Violation Precinct,Violation County,Street Name,Violation Datetime,Violation Datetime (Month),Violation Datetime (Day),Violation Datetime (Year),Violation Datetime (Hour),Violation Rate,Violation Rate (Scaled)
1836,0,0,0,78,K,13TH STREET,2016-06-22 09:34:00,6,22,2016,9,100.000000,1.000000
1844,6430,5280,5430,78,K,13TH STREET,2016-06-22 09:34:00,6,22,2016,9,100.000000,1.000000
1840,0,0,0,78,K,13TH STREET,2016-06-22 09:37:00,6,22,2016,9,33.333333,0.327869
1852,6430,5280,5430,78,K,13TH STREET,2016-06-22 13:03:00,6,22,2016,13,25.000000,0.243852
1849,6430,5130,5280,78,K,13TH STREET,2016-06-28 03:31:00,6,28,2016,3,20.000000,0.193443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14646,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:09:00,9,28,2023,3,9.803922,0.090646
14647,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:11:00,9,28,2023,3,7.843137,0.070878
14648,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:13:00,9,28,2023,3,5.882353,0.051109
14649,6430,5130,5280,78,Kings,13TH STREET,2023-09-28 03:15:00,9,28,2023,3,3.921569,0.031340
